In [20]:

import ee
#import ee.mapclient
from auxil.eeMad import imad
ee.Initialize()


# --------------------------------------------------------------------------------------------------------
# ----------------------------------------------Functions for preparing collections-----------------------

# Study area in Botswana

poly = ee.Geometry.Polygon([23.44,-20.72,26.42,-24.15,29.50,-22.17, 26.10,-18.97,24.91,-18.98, 23.44,-20.72], None, False)
 
   
def iMad(cid,poly,sd1,ed1,sd2,ed2,bns,maxitr):
    collection = ee.ImageCollection(cid) \
         .filterBounds(poly) \
         .filterDate(ee.Date(sd1), ee.Date(ed1)) \
         .sort('system:time_start',True)
    image1 = collection.max().select(bns).clip(poly) 
      
    collection = ee.ImageCollection(cid) \
         .filterBounds(poly) \
         .filterDate(ee.Date(sd2), ee.Date(ed2)) \
         .sort('system:time_start',True)                   
    image2 = collection.max().select(bns).clip(poly) 
    image2 = image2.register(image1,60)
    inputlist = ee.List.sequence(1,maxitr)
    
    first = ee.Dictionary({'done':ee.Number(0),
          'image':image1.addBands(image2),
          'allrhos': [ee.List.sequence(1,len(bns))],
          'chi2':ee.Image.constant(0),
          'MAD':ee.Image.constant(0)}) 
    madnames = ['MAD'+str(i+1) for i in range(len(bns))]
#  run the algorithm    
    result = ee.Dictionary(inputlist.iterate(imad,first))                
    MAD = ee.Image(result.get('MAD')).rename(madnames)
    return MAD

collectionid = 'LANDSAT/LC08/C01/T1_SR'
bandNames = ['B4','B5']
startDate1 = '2016-04-15'
endDate1 = '2016-09-30'
startDate2 = '2018-04-15'
endDate2 = '2018-09-15'
maxitr = 15
MAD = iMad(collectionid,poly,startDate1,
           endDate1,startDate2,endDate2,
           bandNames,maxitr)

# assexportname = 'users/goharghazaryan6/MAD_trails/trail100'
assexportname = 'users/mortcanty/imad/gohar3'
assexport = ee.batch.Export.image.toAsset(MAD.clip(poly),
           description='assetExportTaskMAD', 
           assetId=assexportname,scale=30,maxPixels=1e9)
assexportid = str(assexport.id)
print '****Exporting to Assets, task id: %s'%assexportid
assexport.start()          


****Exporting to Assets, task id: 6LJYPPNGZLFEZBCZWOKYQWJ3
